In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error
from scipy.special import expit  # Sigmoid function for propensity score
from kernel_regression import KernelRegression
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

In [2]:
from sklearn.neural_network import MLPClassifier, MLPRegressor

In [3]:
from pathlib import Path
import os
import glob
from joblib import dump, load
import pandas as pd
import scipy
import scipy.stats
import scipy.special
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from utils.riesznet import RieszNet
from utils.moments import ate_moment_fn
from utils.ihdp_data import *

In [4]:
moment_fn = ate_moment_fn

In [5]:
drop_prob = 0.0  # dropout prob of dropout layers throughout notebook
n_hidden = 100  # width of hidden layers throughout notebook

# Training params
learner_lr = 1e-5
learner_l2 = 1e-3
learner_l1 = 0.0
n_epochs = 600
earlystop_rounds = 40 # how many epochs to wait for an out-of-sample improvement
earlystop_delta = 1e-4
target_reg = 1.0
riesz_weight = 0.1

bs = 64
device = torch.cuda.current_device() if torch.cuda.is_available() else None
print("GPU:", torch.cuda.is_available())

from itertools import chain, combinations
from itertools import combinations_with_replacement as combinations_w_r

def _combinations(n_features, degree, interaction_only):
        comb = (combinations if interaction_only else combinations_w_r)
        return chain.from_iterable(comb(range(n_features), i)
                                   for i in range(0, degree + 1))

class Learner(nn.Module):

    def __init__(self, n_t, n_hidden, p, degree, interaction_only=False):
        super().__init__()
        n_common = 200
        self.monomials = list(_combinations(n_t, degree, interaction_only))
        self.common = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_t, n_common), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_common, n_common), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_common, n_common), nn.ELU())
        self.riesz_nn = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_common, 1))
        self.riesz_poly = nn.Sequential(nn.Linear(len(self.monomials), 1))
        self.reg_nn0 = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_common, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, 1))
        self.reg_nn1 = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_common, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, 1))
        self.reg_poly = nn.Sequential(nn.Linear(len(self.monomials), 1))


    def forward(self, x):
        poly = torch.cat([torch.prod(x[:, t], dim=1, keepdim=True)
                          for t in self.monomials], dim=1)
        feats = self.common(x)
        riesz = self.riesz_nn(feats) + self.riesz_poly(poly)
        reg = self.reg_nn0(feats) * (1 - x[:, [0]]) + self.reg_nn1(feats) * x[:, [0]] + self.reg_poly(poly)
        return torch.cat([reg, riesz], dim=1)

GPU: False


In [6]:
drop_prob = 0.0  # dropout prob of dropout layers throughout notebook
n_hidden = 100  # width of hidden layers throughout notebook

# Training params
learner_lr = 1e-5
learner_l2 = 1e-3
learner_l1 = 0.0
n_epochs = 600
earlystop_rounds = 40 # how many epochs to wait for an out-of-sample improvement
earlystop_delta = 1e-4
target_reg = 1.0
riesz_weight = 0.1

bs = 64
device = torch.cuda.current_device() if torch.cuda.is_available() else None
print("GPU:", torch.cuda.is_available())

from itertools import chain, combinations
from itertools import combinations_with_replacement as combinations_w_r

def _combinations(n_features, degree, interaction_only):
        comb = (combinations if interaction_only else combinations_w_r)
        return chain.from_iterable(comb(range(n_features), i)
                                   for i in range(0, degree + 1))

class Learner(nn.Module):

    def __init__(self, n_t, n_hidden, p, degree, interaction_only=False):
        super().__init__()
        n_common = 200
        self.monomials = list(_combinations(n_t, degree, interaction_only))
        self.common = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_t, n_common), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_common, n_common), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_common, n_common), nn.ELU())
        self.riesz_nn = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_common, 1))
        self.riesz_poly = nn.Sequential(nn.Linear(len(self.monomials), 1))
        self.reg_nn0 = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_common, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, 1))
        self.reg_nn1 = nn.Sequential(nn.Dropout(p=p), nn.Linear(n_common, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, n_hidden), nn.ELU(),
                                    nn.Dropout(p=p), nn.Linear(n_hidden, 1))
        self.reg_poly = nn.Sequential(nn.Linear(len(self.monomials), 1))


    def forward(self, x):
        poly = torch.cat([torch.prod(x[:, t], dim=1, keepdim=True)
                          for t in self.monomials], dim=1)
        feats = self.common(x)
        riesz = self.riesz_nn(feats) + self.riesz_poly(poly)
        reg = self.reg_nn0(feats) * (1 - x[:, [0]]) + self.reg_nn1(feats) * x[:, [0]] + self.reg_poly(poly)
        return torch.cat([reg, riesz], dim=1)

GPU: False


In [7]:
# Simulation parameters
n = 3000 # Number of samples
p = 10   # Number of covariates
treatment_effect = 5.0  # True treatment effect

# Generate covariates
np.random.seed(0)

In [8]:
true_ATE = treatment_effect

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class NeuralNetBiasCorrection:
    def __init__(self, input_dim, hidden_dim=100, max_iter=600, tol=1e-4, lbd=0.01, loss="Logit", lr=0.01):
        """
        ニューラルネットワークを用いたバイアス補正モデル
        :param input_dim: 入力特徴量の次元
        :param hidden_dim: 隠れ層のユニット数
        :param max_iter: 最大反復回数
        :param tol: 収束許容誤差
        :param lbd: 正則化項の重み
        :param loss: 損失関数の種類（"Logit", "DBCLS", "DBCLogit"）
        :param lr: 学習率
        """
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.max_iter = max_iter
        self.tol = tol
        self.lbd = lbd
        self.lr = lr
        
        # ニューラルネットワークの構築
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
        
        if loss == "Logit":
            self.criterion = self._logistic_loss_function
        elif loss == "CBPS":
            self.criterion = self._cbps_loss_function
        elif loss == "DBCLS":
            self.criterion = self._least_squares_loss_function
        elif loss == "DBCUKL":
            self.criterion = self._ukl_loss_function
        else:
            raise ValueError("Invalid loss function specified")

        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        
    
    def _least_squares_loss_function(self, X_tensor, outputs, targets):
        """最小二乗誤差損失関数"""
        outputs = torch.clamp(outputs, min=0.05, max=0.95)
        loss = torch.mean(-2 * (1 / outputs + 1 / (1 - outputs)) + (targets / outputs - (1 - targets) / (1 - outputs))**2)
        return loss + self.lbd * self._l2_regularization()
    
    def _ukl_loss_function(self, X_tensor, outputs, targets):
        """制約付きロジスティック損失関数"""
        outputs = torch.clamp(outputs, min=0.05, max=0.95)
        # Correcting the formula with the proper variable names
        loss = - torch.log(1/outputs) - torch.log(1/(1 - outputs)) + targets / outputs + (1 - targets) / (1 - outputs)
        # Mean loss calculation
        loss = torch.mean(loss)
        return loss + self.lbd * self._l2_regularization()
    
    def _logistic_loss_function(self, X_tensor, outputs, targets):
        """ロジスティック損失関数"""
        outputs = torch.clamp(outputs, min=0.05, max=0.95)
        loss = nn.BCELoss()(outputs, targets)  # バイナリクロスエントロピー
        return loss + self.lbd * self._l2_regularization()
    
    def _cbps_loss_function(self, X_tensor, outputs, targets):
        """ロジスティック損失関数"""
        outputs = torch.clamp(outputs, min=0.05, max=0.95)
        loss =  torch.mean((targets * X_tensor / outputs - (1 - targets) * X_tensor / (1 - outputs))**2, axis=0)
        loss = (loss**2).mean()
        return loss + self.lbd * self._l2_regularization()

    def _l2_regularization(self):
        """L2正則化項の計算"""
        reg_loss = sum(torch.sum(param**2) for param in self.model.parameters())
        return reg_loss
    
    def fit(self, X, T):
        """
        モデルの学習
        :param X: 説明変数（N×d の配列）
        :param T: 目的変数（N×1 のバイナリ配列）
        """
        X_tensor = torch.tensor(X, dtype=torch.float32)
        T_tensor = torch.tensor(T, dtype=torch.float32).view(-1, 1)

        prev_loss = float('inf')
        for epoch in range(self.max_iter):
            self.optimizer.zero_grad()
            outputs = self.model(X_tensor)
            loss = self.criterion(X_tensor, outputs, T_tensor)
            loss.backward()
            self.optimizer.step()

            # 収束判定
            if abs(prev_loss - loss.item()) < self.tol:
                break
            prev_loss = loss.item()

    def predict_proba(self, X):
        """
        予測確率の計算
        :param X: 説明変数（N×d の配列）
        :return: クラス1の確率
        """
        X_tensor = torch.tensor(X, dtype=torch.float32)
        with torch.no_grad():
            probas = torch.clamp(self.model(X_tensor), min=0.05, max=0.95).numpy()
        return np.hstack([1 - probas, probas])

    def predict(self, X):
        """
        クラスラベルの予測
        :param X: 説明変数（N×d の配列）
        :return: 0または1の予測クラス
        """
        probas = self.predict_proba(X)
        return (probas[:, 1] >= 0.5).astype(int)

    def get_params(self):
        """
        学習済みパラメータの取得
        :return: モデルの重みとバイアス
        """
        params = {name: param.detach().numpy() for name, param in self.model.named_parameters()}
        return params

### Data generation

In [10]:
num_trial = 1000

In [11]:
nsims = 1000
np.random.seed(123)
#sim_ids = np.random.choice(len(simulation_files), nsims, replace=False)
methods = ['dr', 'direct', 'ips']
srr = {'dr' : True, 'direct' : False, 'ips' : True}


In [13]:
result_list = []
result_list2 = []

for tr in range(num_trial):
    result_list_temp = []

    X = np.random.normal(0, 1, (n, p))

    # Define a propensity score model
    # Assume treatment probability is a sigmoid function of a subset of covariates
    X_temp = np.concatenate([X, X**2, np.array([X[:, 0]*X[:, 1], X[:, 1]*X[:, 2], X[:, 0]*X[:, 2]]).T], axis=1)
    propensity_coef = np.random.normal(0, 0.5, X_temp.shape[1])
    propensity_scores = expit(X_temp @ propensity_coef)  # Calculate propensity scores

    # Generate treatment assignment based on propensity scores
    T = np.random.binomial(1, propensity_scores)

    # Generate outcome with treatment effect
    # Assume a simple linear model for demonstration
    beta = np.random.normal(0, 1, p)
    Y = X@ beta + (X @ beta)**2 + 3*np.sin(X[:, 0]) + 1.1 + treatment_effect * T + np.random.normal(0, 1, n)

    X_treatment = X[T == 1]
    X_control = X[T == 0]
    
    y_scaler = StandardScaler(with_mean=True).fit(np.array([Y]).T)
    y = y_scaler.transform(np.array([Y]).T)
    XT = np.c_[T, X]
    
    X_train, X_test, y_train, y_test = train_test_split(XT, y, test_size = 0.2)

    torch.cuda.empty_cache()
    learner = Learner(X_train.shape[1], n_hidden, drop_prob, 0, interaction_only=True)
    agmm = RieszNet(learner, moment_fn)
    # Fast training
    agmm.fit(X_train, y_train, Xval=X_test, yval=y_test,
             earlystop_rounds=2, earlystop_delta=earlystop_delta,
             learner_lr=1e-4, learner_l2=learner_l2, learner_l1=learner_l1,
             n_epochs=100, bs=bs, target_reg=target_reg,
             riesz_weight=riesz_weight, optimizer='adam',
             model_dir=str(Path.home()), device=device, verbose=0)
    # Fine tune
    agmm.fit(X_train, y_train, Xval=X_test, yval=y_test,
             earlystop_rounds=earlystop_rounds, earlystop_delta=earlystop_delta,
             learner_lr=learner_lr, learner_l2=learner_l2, learner_l1=learner_l1,
             n_epochs=600, bs=bs, target_reg=target_reg,
             riesz_weight=riesz_weight, optimizer='adam', warm_start=True,
             model_dir=str(Path.home()), device=device, verbose=0)
    
    params = tuple(x * y_scaler.scale_[0] for method in methods
                   for x in agmm.predict_avg_moment(XT, y,  model='earlystop', method = method, srr = srr[method])) + (true_ATE, )
                        
    result_list2.append(params)
    
    Y = y.T[0]
    
    Y_treatment = Y[T == 1]
    Y_control = Y[T == 0]
    
    for method in ["DBCLS", "DBCUKL", "CBPS", "Logit"]:
        prop_model = NeuralNetBiasCorrection(input_dim=p, lbd = 0.01, loss=method)
        prop_model.fit(X, T)
        est_prop_score = prop_model.predict_proba(X)[:, 1]
        est_prop_score_dbc = est_prop_score

        #treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
        #control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
        
        treatment_outcome_model = MLPRegressor(random_state=1, max_iter=600)
        control_outcome_model = MLPRegressor(random_state=1, max_iter=600)

        treatment_outcome_model.fit(X_treatment, Y_treatment)
        control_outcome_model.fit(X_control, Y_control)

        est_treatment_outcome = treatment_outcome_model.predict(X)
        est_control_outcome = control_outcome_model.predict(X)

        IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

        # Evaluate performance
        IPW_bias = IPW_est - true_ATE

        result_list_temp.append(IPW_est)

        DM_est = np.mean(est_treatment_outcome - est_control_outcome)

        # Evaluate performance
        DM_bias = DM_est - true_ATE
        
        result_list_temp.append(DM_est)

        DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

        # Evaluate performance
        DR_bias = DR_est - true_ATE

        result_list_temp.append(DR_est)
    
    ##### Linear models
    
    # Fit a linear model to estimate the treatment effect
    model = LinearRegression()
    model.fit(np.hstack([X, T.reshape(-1, 1)]), Y)
    estimated_treatment_effect = model.coef_[-1]

    # Evaluate performance
    true_ATE = treatment_effect
    bias = estimated_treatment_effect - true_ATE
    mse = mean_squared_error(Y, model.predict(np.hstack([X, T.reshape(-1, 1)])))

    result_list_temp.append(estimated_treatment_effect)
    
    #### CBPS
    
    # Enable automatic conversion of Pandas DataFrame to R DataFrame
    pandas2ri.activate()

    # Simulate data in Python

    # Create a pandas DataFrame
    column_names = [f'X{i+1}' for i in range(p)]
    df = pd.DataFrame(X, columns=column_names)
    df['T'] = T
    df['Y'] = Y


    # Convert pandas DataFrame to R DataFrame
    r_df = pandas2ri.py2rpy(df)

    ro.r.assign("p", p)

    # Load the CBPS package in R and fit the model for ATE estimation
    ro.r('''
        library(CBPS)
        estimate_cbps_ate <- function(df) {
            formula_str <- paste("T ~", paste(names(df)[1:{p}], collapse=" + "))

            # CBPSの適用 (ATEの推定、ATT=0)
            model <- CBPS(as.formula(formula_str), data = df, ATT = 0, method = "exact")

            # 推定された傾向スコアの取得
            df$propensity_score <- fitted(model)

            # IPW (Inverse Probability Weighting) を適用
            df$weight <- ifelse(df$T == 1, 1 / df$propensity_score, 1 / (1 - df$propensity_score))

            # 重み付き回帰によるATEの推定
            result <- lm(Y ~ T, data = df, weights = df$weight)

            return(df$propensity_score)
        }
    ''')

    # R関数を呼び出してATEと傾向スコアを取得
    est_prop_score = ro.r['estimate_cbps_ate'](r_df)
    
    est_prop_score_cbps = est_prop_score
    
    #print(er)

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE
    
    result_list_temp.append(DR_est)
    
    result_list_temp = np.array(result_list_temp)*y_scaler.scale_[0]
    
    result_list.append(result_list_temp)
    
    res = tuple(np.array(x) for x in zip(*result_list2))
    truth = res[-1:]
    res_dict = {}

    res_list_temp = []
    for it, method in enumerate(methods):
        point, lb, ub = res[it * 3: (it + 1)*3]
        res_list_temp.append(point)
        
    result_list_final = np.concatenate([np.array(result_list), np.array(res_list_temp).T], axis=1)
    
    print(np.round(np.sqrt(np.mean((result_list_final - true_ATE)**2, axis=0)), 3))

/opt/anaconda3/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/Users/mkato/Dropbox/ATEEstimation/ATEEstimation/utils/riesznet.py:247: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use cas

[0.8   0.217 0.114 0.174 0.217 0.13  1.381 0.217 0.122 0.288 0.217 0.116
 0.612 0.927 0.747 0.752 0.051 0.104 0.133]


/opt/anaconda3/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/Users/mkato/Dropbox/ATEEstimation/ATEEstimation/utils/riesznet.py:247: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use cas

[0.875 0.247 0.126 0.374 0.247 0.129 1.652 0.247 0.128 0.39  0.247 0.119
 1.021 1.102 1.117 1.12  0.372 0.452 0.129]


/Users/mkato/Dropbox/ATEEstimation/ATEEstimation/utils/riesznet.py:247: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(os.path.join(self.model_dir,
/Users/m

...

/Users/mkato/Dropbox/ATEEstimation/ATEEstimation/utils/riesznet.py:247: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(os.path.join(self.model_dir,
/Users/m

[1.467 0.694 0.667 1.759 0.694 0.67  1.978 0.694 0.693 2.523 0.694 0.692
 2.659 2.886 2.161 2.164 0.666 0.66  0.636]


/opt/anaconda3/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/Users/mkato/Dropbox/ATEEstimation/ATEEstimation/utils/riesznet.py:247: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use cas

[1.466 0.694 0.667 1.757 0.694 0.67  1.976 0.694 0.692 2.52  0.694 0.691
 2.656 2.883 2.159 2.163 0.665 0.659 0.638]


Exception ignored in: <function SexpCapsule.__del__ at 0x174f84f40>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/rpy2/rinterface_lib/_rinterface_capi.py", line 141, in __del__
    def __del__(self):

KeyboardInterrupt: 

KeyboardInterrupt



In [15]:
res = tuple(np.array(x) for x in zip(*result_list2))
truth = res[-1:]
res_dict = {}

res_list_temp = []
for it, method in enumerate(methods):
    point, lb, ub = res[it * 3: (it + 1)*3]
    res_list_temp.append(point)

In [16]:
result_list_final = np.concatenate([np.array(result_list), np.array(res_list_temp).T], axis=1)

In [17]:
np.savetxt("ATE_result4.csv", result_list_final)

In [18]:
np.round(np.sqrt(np.mean((result_list_final - true_ATE)**2, axis=0)), 3)

array([1.466, 0.694, 0.667, 1.757, 0.694, 0.67 , 1.976, 0.694, 0.692,
       2.52 , 0.694, 0.691, 2.656, 2.883, 2.159, 2.163, 0.665, 0.659,
       0.638])

In [14]:
len(result_list_final)

446

In [23]:
np.array(result_list)

array([[2.33420245, 2.69248963, 2.78836369, 2.05049704, 2.69248963,
        2.78292243, 1.17453   , 2.69248963, 2.72538244, 1.14573749,
        2.69248963, 2.73356029, 0.1674897 , 0.17318048, 2.18604934,
        2.17192884],
       [2.90344812, 3.01980062, 3.00719726, 2.72691919, 3.01980062,
        3.00100208, 2.71324345, 3.01980062, 3.00840853, 2.78409739,
        3.01980062, 2.99738415, 2.88539763, 2.76368037, 2.96821651,
        2.95634584]])

In [49]:
res = tuple(np.array(x) for x in zip(*result_list2))
truth = res[-1:]
res_dict = {}

res_list_temp = []
for it, method in enumerate(methods):
    point, lb, ub = res[it * 3: (it + 1)*3]
    res_list_temp.append(point)

In [65]:
np.concatenate([np.array(result_list), np.array(res_list_temp).T], axis=1)

array([[3.13824887, 2.98757769, 3.00656468, 2.93689259, 2.98757769,
        2.99441305, 3.21834824, 2.98757769, 2.99590128, 3.08447467,
        2.98757769, 2.99751422, 2.96991877, 2.37510366, 2.98757769,
        2.98152865, 2.98207014, 2.99127682, 2.94388743],
       [3.51725208, 3.55584463, 3.4887395 , 3.46519865, 3.55584463,
        3.50424109, 5.49242025, 3.55584463, 3.54794203, 4.19640302,
        3.55584463, 3.53464643, 4.90452631, 4.88247822, 3.55584463,
        3.56359317, 3.10494999, 3.1386278 , 3.01535168]])

In [60]:
np.array(res_list_temp).T.shape

(2, 3)

In [46]:
res_dict

{'dr': {'point': array([2.98207014, 3.10494999]),
  'lb': array([2.89669808, 3.02416755]),
  'ub': array([3.0674422 , 3.18573244]),
  'cov': 0.5,
  'bias': 0.043510066762107336},
 'direct': {'point': array([2.99127682, 3.1386278 ]),
  'lb': array([2.96516755, 3.11051796]),
  'ub': array([3.01738608, 3.16673764]),
  'cov': 0.5,
  'bias': 0.06495230559765441},
 'ips': {'point': array([2.94388743, 3.01535168]),
  'lb': array([2.29009494, 2.70514945]),
  'ub': array([3.59767992, 3.3255539 ]),
  'cov': 1.0,
  'bias': -0.020380447898789456}}

In [58]:
np.array(result_list).shape

(2, 16)

In [186]:
result_list = []

for tr in range(num_trial):
    result_list_temp = []
    
    
    X = np.random.normal(0, 1, (n, p))

    # Define a propensity score model
    # Assume treatment probability is a sigmoid function of a subset of covariates
    X_temp = np.concatenate([X, X**2, np.array([X[:, 0]*X[:, 1], X[:, 1]*X[:, 2], X[:, 0]*X[:, 2]]).T], axis=1)
    propensity_coef = np.random.normal(0, 0.5, X_temp.shape[1])
    propensity_scores = expit(X_temp @ propensity_coef)  # Calculate propensity scores

    # Generate treatment assignment based on propensity scores
    T = np.random.binomial(1, propensity_scores)

    # Generate outcome with treatment effect
    # Assume a simple linear model for demonstration
    beta = np.random.normal(0, 1, p)
    Y = (X @ beta)**2 + 1.1 + treatment_effect * T + np.random.normal(0, 1, n)

    X_treatment = X[T == 1]
    X_control = X[T == 0]

    Y_treatment = Y[T == 1]
    Y_control = Y[T == 0]
    
    #### Direct bias correction
    prop_model = NeuralNetBiasCorrection(input_dim=p, lbd = 0.01, loss="DBCLS")
    prop_model.fit(X, T)
    est_prop_score = prop_model.predict_proba(X)[:, 1]
    est_prop_score_dbc = est_prop_score

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    
    #### Direct bias correction
    prop_model = NeuralNetBiasCorrection(input_dim=p, lbd = 0.01, loss="CBPS")
    prop_model.fit(X, T)
    est_prop_score = prop_model.predict_proba(X)[:, 1]
    est_prop_score_dbc = est_prop_score

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    
    ##### Linear models
    
    # Fit a linear model to estimate the treatment effect
    model = LinearRegression()
    model.fit(np.hstack([X, T.reshape(-1, 1)]), Y)
    estimated_treatment_effect = model.coef_[-1]

    # Evaluate performance
    true_ATE = treatment_effect
    bias = estimated_treatment_effect - true_ATE
    mse = mean_squared_error(Y, model.predict(np.hstack([X, T.reshape(-1, 1)])))

    print(f"Estimated ATE: {estimated_treatment_effect}")
    print(f"Bias: {bias}")
    print(f"Mean Squared Error: {mse}")

    result_list_temp.append(estimated_treatment_effect)
    
    #### Logistc regression 
    
    prop_model = NeuralNetBiasCorrection(input_dim=p, lbd = 0., loss="Logit")
    prop_model.fit(X, T)
    est_prop_score = prop_model.predict_proba(X)[:, 1]

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    
    #### CBPS
    
    # Enable automatic conversion of Pandas DataFrame to R DataFrame
    pandas2ri.activate()

    # Simulate data in Python

    # Create a pandas DataFrame
    column_names = [f'X{i+1}' for i in range(p)]
    df = pd.DataFrame(X, columns=column_names)
    df['T'] = T
    df['Y'] = Y


    # Convert pandas DataFrame to R DataFrame
    r_df = pandas2ri.py2rpy(df)

    ro.r.assign("p", p)

    # Load the CBPS package in R and fit the model for ATE estimation
    ro.r('''
        library(CBPS)
        estimate_cbps_ate <- function(df) {
            formula_str <- paste("T ~", paste(names(df)[1:{p}], collapse=" + "))

            # CBPSの適用 (ATEの推定、ATT=0)
            model <- CBPS(as.formula(formula_str), data = df, ATT = 0, method = "exact")

            # 推定された傾向スコアの取得
            df$propensity_score <- fitted(model)

            # IPW (Inverse Probability Weighting) を適用
            df$weight <- ifelse(df$T == 1, 1 / df$propensity_score, 1 / (1 - df$propensity_score))

            # 重み付き回帰によるATEの推定
            result <- lm(Y ~ T, data = df, weights = df$weight)

            return(df$propensity_score)
        }
    ''')

    # R関数を呼び出してATEと傾向スコアを取得
    est_prop_score = ro.r['estimate_cbps_ate'](r_df)
    
    est_prop_score_cbps = est_prop_score
    
    #print(er)

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    
    
    result_list.append(result_list_temp)

Estimated ATE: 3.666468677704666
Bias: 0.6664686777046658
Estimated ATE: 3.197314788090588
Bias: 0.1973147880905879
Estimated ATE: 3.1961531208078506
Bias: 0.19615312080785063
Estimated ATE: 4.601117985662866
Bias: 1.6011179856628663
Estimated ATE: 3.197314788090588
Bias: 0.1973147880905879
Estimated ATE: 3.21044721347009
Bias: 0.21044721347008988
Estimated ATE: 3.3286003541844162
Bias: 0.32860035418441624
Mean Squared Error: 17.199976092090044
Estimated ATE: 3.5808537722716682
Bias: 0.5808537722716682
Estimated ATE: 3.197314788090588
Bias: 0.1973147880905879
Estimated ATE: 3.194816434072869
Bias: 0.19481643407286908
Estimated ATE: 3.3077861256136964
Bias: 0.30778612561369645
Estimated ATE: 3.197314788090588
Bias: 0.1973147880905879
Estimated ATE: 3.2033339822009474
Bias: 0.20333398220094745
Estimated ATE: 3.1291984672441298
Bias: 0.12919846724412976
Estimated ATE: 2.9329038786413597
Bias: -0.06709612135864029
Estimated ATE: 2.957972282635173
Bias: -0.04202771736482713
Estimated ATE: 3

Estimated ATE: 4.480063581477346
Bias: 1.4800635814773457
Estimated ATE: 3.211012872255794
Bias: 0.21101287225579402
Estimated ATE: 3.1660743637380966
Bias: 0.1660743637380966
Estimated ATE: 2.4853513152555715
Bias: -0.5146486847444285
Mean Squared Error: 15.831362221934224
Estimated ATE: 3.3761699274878327
Bias: 0.37616992748783273
Estimated ATE: 3.211012872255794
Bias: 0.21101287225579402
Estimated ATE: 3.174281498886304
Bias: 0.17428149888630395
Estimated ATE: 2.578085457490623
Bias: -0.4219145425093771
Estimated ATE: 3.211012872255794
Bias: 0.21101287225579402
Estimated ATE: 3.113174035127961
Bias: 0.11317403512796087
Estimated ATE: 2.9283066741281996
Bias: -0.07169332587180044
Estimated ATE: 3.042952440901091
Bias: 0.04295244090109085
Estimated ATE: 3.0299444571977516
Bias: 0.02994445719775163
Estimated ATE: 2.549836212862722
Bias: -0.4501637871372779
Estimated ATE: 3.042952440901091
Bias: 0.04295244090109085
Estimated ATE: 3.037379304998199
Bias: 0.03737930499819919
Estimated ATE

Estimated ATE: 3.361435236237726
Bias: 0.361435236237726
Estimated ATE: 3.3697215452767826
Bias: 0.3697215452767826
Estimated ATE: 3.319099506579724
Bias: 0.31909950657972397
Estimated ATE: 4.534702122759686
Bias: 1.5347021227596862
Estimated ATE: 3.3697215452767826
Bias: 0.3697215452767826
Estimated ATE: 3.3735313947034604
Bias: 0.3735313947034604
Estimated ATE: 3.231489395949986
Bias: 0.23148939594998597
Estimated ATE: 3.8223027174408695
Bias: 0.8223027174408695
Estimated ATE: 3.7102635797078722
Bias: 0.7102635797078722
Estimated ATE: 7.016509392524352
Bias: 4.016509392524352
Estimated ATE: 3.8223027174408695
Bias: 0.8223027174408695
Estimated ATE: 3.8036748897198662
Bias: 0.8036748897198662
Estimated ATE: 7.088392945419314
Bias: 4.088392945419314
Mean Squared Error: 59.38350448675715
Estimated ATE: 4.262228979372108
Bias: 1.2622289793721082
Estimated ATE: 3.8223027174408695
Bias: 0.8223027174408695
Estimated ATE: 3.7282271290617235
Bias: 0.7282271290617235
Estimated ATE: 7.032412773

Estimated ATE: 8.741429384382585
Bias: 5.7414293843825845
Estimated ATE: 4.017896885419382
Bias: 1.0178968854193817
Estimated ATE: 4.020466890611351
Bias: 1.0204668906113508
Estimated ATE: 2.3983881022073796
Bias: -0.6016118977926204
Estimated ATE: 2.5349538900006725
Bias: -0.4650461099993275
Estimated ATE: 2.6917158841836373
Bias: -0.3082841158163627
Estimated ATE: 1.911896267480981
Bias: -1.088103732519019
Estimated ATE: 2.5349538900006725
Bias: -0.4650461099993275
Estimated ATE: 2.6186504261357295
Bias: -0.3813495738642705
Estimated ATE: 1.8105533280209705
Bias: -1.1894466719790295
Mean Squared Error: 6.351877588816076
Estimated ATE: 2.4779278066888395
Bias: -0.5220721933111605
Estimated ATE: 2.5349538900006725
Bias: -0.4650461099993275
Estimated ATE: 2.696285958602931
Bias: -0.30371404139706915
Estimated ATE: 1.7560116381390043
Bias: -1.2439883618609957
Estimated ATE: 2.5349538900006725
Bias: -0.4650461099993275
Estimated ATE: 2.5931690073033575
Bias: -0.40683099269664247
Estimated

Estimated ATE: 4.320964480125358
Bias: 1.3209644801253582
Estimated ATE: 2.9509287823447377
Bias: -0.049071217655262345
Estimated ATE: 3.01881200736428
Bias: 0.018812007364279903
Estimated ATE: 4.100405569002077
Bias: 1.1004055690020769
Estimated ATE: 2.9509287823447377
Bias: -0.049071217655262345
Estimated ATE: 2.9947901591878034
Bias: -0.005209840812196553
Estimated ATE: 2.1863522079864794
Bias: -0.8136477920135206
Mean Squared Error: 89.10015774022669
Estimated ATE: 3.4903545864797114
Bias: 0.4903545864797114
Estimated ATE: 2.9509287823447377
Bias: -0.049071217655262345
Estimated ATE: 2.9996107426101135
Bias: -0.00038925738988648817
Estimated ATE: 2.38228835952334
Bias: -0.61771164047666
Estimated ATE: 2.9509287823447377
Bias: -0.049071217655262345
Estimated ATE: 2.9732482794736774
Bias: -0.026751720526322575
Estimated ATE: 3.354539764877999
Bias: 0.3545397648779991
Estimated ATE: 2.733476919144341
Bias: -0.2665230808556589
Estimated ATE: 2.7709241344686806
Bias: -0.2290758655313194

Estimated ATE: 3.803234638640029
Bias: 0.8032346386400291
Estimated ATE: 3.0236413163437925
Bias: 0.023641316343792518
Estimated ATE: 2.9631953568719727
Bias: -0.03680464312802734
Estimated ATE: 2.854627946016926
Bias: -0.14537205398307407
Mean Squared Error: 7.055476810045092
Estimated ATE: 3.059604597465503
Bias: 0.05960459746550306
Estimated ATE: 3.0236413163437925
Bias: 0.023641316343792518
Estimated ATE: 2.9534848687660142
Bias: -0.04651513123398576
Estimated ATE: 2.868203055891288
Bias: -0.13179694410871212
Estimated ATE: 3.0236413163437925
Bias: 0.023641316343792518
Estimated ATE: 2.9438196787544353
Bias: -0.05618032124556471
Estimated ATE: 3.0342841562223923
Bias: 0.03428415622239234
Estimated ATE: 3.1711169219366946
Bias: 0.1711169219366946
Estimated ATE: 3.1228324165274914
Bias: 0.12283241652749144
Estimated ATE: 3.344726412093466
Bias: 0.3447264120934661
Estimated ATE: 3.1711169219366946
Bias: 0.1711169219366946
Estimated ATE: 3.155076565312471
Bias: 0.15507656531247083
Esti

Estimated ATE: 2.7555058982652807
Bias: -0.2444941017347193
Estimated ATE: 2.782118738278893
Bias: -0.21788126172110678
Estimated ATE: 2.839058984717843
Bias: -0.1609410152821571
Estimated ATE: 2.196029496876571
Bias: -0.8039705031234292
Estimated ATE: 2.782118738278893
Bias: -0.21788126172110678
Estimated ATE: 2.7635034041581306
Bias: -0.23649659584186944
Estimated ATE: 2.7016345063492233
Bias: -0.2983654936507767
Estimated ATE: 2.8629015075726243
Bias: -0.1370984924273757
Estimated ATE: 2.9360734029668585
Bias: -0.06392659703314152
Estimated ATE: 1.2658518190115575
Bias: -1.7341481809884425
Estimated ATE: 2.8629015075726243
Bias: -0.1370984924273757
Estimated ATE: 2.8823815331070146
Bias: -0.11761846689298538
Estimated ATE: 2.827482605718764
Bias: -0.17251739428123614
Mean Squared Error: 1.3058491802496193
Estimated ATE: 3.2082817997199164
Bias: 0.20828179971991645
Estimated ATE: 2.8629015075726243
Bias: -0.1370984924273757
Estimated ATE: 2.930958519731527
Bias: -0.0690414802684729
E

Estimated ATE: -7.626616994626293
Bias: -10.626616994626293
Estimated ATE: 0.5493879170123054
Bias: -2.4506120829876945
Estimated ATE: 0.5214340880225721
Bias: -2.478565911977428
Estimated ATE: 3.174375735416997
Bias: 0.17437573541699702
Estimated ATE: 3.171624034669089
Bias: 0.17162403466908893
Estimated ATE: 3.138903961513802
Bias: 0.1389039615138019
Estimated ATE: 3.3970353804063835
Bias: 0.39703538040638353
Estimated ATE: 3.171624034669089
Bias: 0.17162403466908893
Estimated ATE: 3.1694581143442555
Bias: 0.1694581143442555
Estimated ATE: 3.312909791460142
Bias: 0.3129097914601422
Mean Squared Error: 3.1748817727896337
Estimated ATE: 3.2690252617101723
Bias: 0.26902526171017227
Estimated ATE: 3.171624034669089
Bias: 0.17162403466908893
Estimated ATE: 3.1462490493555046
Bias: 0.14624904935550465
Estimated ATE: 3.3674646443363865
Bias: 0.3674646443363865
Estimated ATE: 3.171624034669089
Bias: 0.17162403466908893
Estimated ATE: 3.1736035353975205
Bias: 0.1736035353975205
Estimated ATE:

Estimated ATE: 3.1958239372274857
Bias: 0.1958239372274857
Estimated ATE: 3.3683085878248966
Bias: 0.3683085878248966
Estimated ATE: 3.3244554932132058
Bias: 0.32445549321320577
Estimated ATE: 3.880025474338841
Bias: 0.8800254743388409
Estimated ATE: 3.3683085878248966
Bias: 0.3683085878248966
Estimated ATE: 3.3560480929849485
Bias: 0.35604809298494855
Estimated ATE: 4.469033708084538
Bias: 1.4690337080845381
Mean Squared Error: 17.06598118618572
Estimated ATE: 3.429115238798844
Bias: 0.42911523879884417
Estimated ATE: 3.3683085878248966
Bias: 0.3683085878248966
Estimated ATE: 3.3300998188626822
Bias: 0.33009981886268225
Estimated ATE: 4.893334277708078
Bias: 1.8933342777080782
Estimated ATE: 3.3683085878248966
Bias: 0.3683085878248966
Estimated ATE: 3.3801812939730254
Bias: 0.38018129397302536
Estimated ATE: 2.6432255099688815
Bias: -0.3567744900311185
Estimated ATE: 2.9536208665436923
Bias: -0.04637913345630773
Estimated ATE: 2.9445478989458653
Bias: -0.05545210105413467
Estimated AT

Estimated ATE: 2.3965091235997074
Bias: -0.6034908764002926
Estimated ATE: 2.9192381457567183
Bias: -0.08076185424328175
Estimated ATE: 2.90939445182553
Bias: -0.09060554817446986
Estimated ATE: 2.893111902677576
Bias: -0.10688809732242399
Mean Squared Error: 1.0366134744464026
Estimated ATE: 2.705623064284074
Bias: -0.2943769357159258
Estimated ATE: 2.9192381457567183
Bias: -0.08076185424328175
Estimated ATE: 2.920454204148047
Bias: -0.0795457958519532
Estimated ATE: 2.8721781237812647
Bias: -0.1278218762187353
Estimated ATE: 2.9192381457567183
Bias: -0.08076185424328175
Estimated ATE: 2.8956861608843663
Bias: -0.10431383911563374
Estimated ATE: 1.0695797376900622
Bias: -1.9304202623099378
Estimated ATE: 2.371528751131591
Bias: -0.6284712488684092
Estimated ATE: 2.386074794911927
Bias: -0.6139252050880728
Estimated ATE: -0.4281587349993001
Bias: -3.4281587349993
Estimated ATE: 2.371528751131591
Bias: -0.6284712488684092
Estimated ATE: 2.355808554015459
Bias: -0.644191445984541
Estimat

In [187]:
np.round(np.sqrt(np.mean((np.array(result_list) - true_ATE)**2, axis=0)), 3)

array([0.813, 0.549, 0.485, 2.61 , 0.549, 0.541, 2.058, 1.126, 0.549,
       0.499, 2.168, 0.549, 0.555])

In [28]:
res = tuple(np.array(x) for x in zip(*result_list2))
truth = res[-1:]
res_dict = {}
for it, method in enumerate(methods):
    point, lb, ub = res[it * 3: (it + 1)*3]
    res_dict[method] = {'point': point, 'lb': lb, 'ub': ub,
                        'cov': np.mean(np.logical_and(truth >= lb, truth <= ub)),
                        'bias': np.mean(point - truth),
                        'rmse': rmse_fn(point, truth)
                        }
    print("{} : bias = {:.3f}, rmse = {:.3f}, cov = {:.3f}".format(method, res_dict[method]['bias'], res_dict[method]['rmse'], res_dict[method]['cov']))

NameError: name 'rmse_fn' is not defined

In [178]:
np.round(np.sqrt(np.mean((np.array(result_list) - true_ATE)**2, axis=0)), 3)

array([0.979, 0.556, 0.503, 2.769, 0.556, 0.555, 1.681, 0.813, 0.556,
       0.513, 1.841, 0.556, 0.561])

In [137]:
np.concatenate([X, X**2], axis=1).shape

(3000, 6)

In [138]:
X_temp

array([[ 9.25924616e-01, -2.03031180e+00,  1.27520423e+00,
         8.57336395e-01,  4.12216601e+00,  1.62614582e+00],
       [-5.66775343e-01,  1.07887815e+00, -9.96268834e-01,
         3.21234289e-01,  1.16397806e+00,  9.92551589e-01],
       [-1.14120031e+00, -1.03892180e-01,  2.86252848e-01,
         1.30233815e+00,  1.07935851e-02,  8.19406930e-02],
       ...,
       [-3.20993243e-01,  1.19378072e+00, -9.92688808e-01,
         1.03036662e-01,  1.42511242e+00,  9.85431070e-01],
       [ 3.32821056e-01,  5.35663125e-02, -1.35952125e+00,
         1.10769855e-01,  2.86934983e-03,  1.84829802e+00],
       [-3.07757600e-01, -1.46619960e+00, -2.25171990e+00,
         9.47147402e-02,  2.14974127e+00,  5.07024251e+00]])

In [139]:
np.max(propensity_scores)

0.9999342940986149

In [140]:
np.min(propensity_scores)

0.47356072751750583

In [141]:
np.max(est_prop_score_dbc)

0.7247125

In [142]:
np.min(est_prop_score_dbc)

0.5067347

In [143]:
np.max(est_prop_score_cbps)

0.86754426622205

In [144]:
np.min(est_prop_score_cbps)

0.3840582792049322

In [145]:
np.round(np.sqrt(np.mean((np.array(result_list) - true_ATE)**2, axis=0)), 3)

array([0.81 , 0.584, 0.531, 4.821, 0.584, 0.607, 1.801, 0.939, 0.584,
       0.55 , 2.265, 0.584, 0.596])

In [175]:
print(f"True Average Treatment Effect (ATE): {true_ATE}")

True Average Treatment Effect (ATE): 3.0


In [176]:
# Fit a linear model to estimate the treatment effect
model = LinearRegression()
model.fit(np.hstack([X, T.reshape(-1, 1)]), Y)
estimated_treatment_effect = model.coef_[-1]

# Evaluate performance
true_ATE = treatment_effect
bias = estimated_treatment_effect - true_ATE
mse = mean_squared_error(Y, model.predict(np.hstack([X, T.reshape(-1, 1)])))

print(f"Estimated ATE: {estimated_treatment_effect}")
print(f"Bias: {bias}")
print(f"Mean Squared Error: {mse}")





Estimated ATE: 2.972625870114331
Bias: -0.027374129885668896
Mean Squared Error: 1.7797254450419215


In [177]:
# Fit a linear model to estimate the treatment effect
prop_model = LogisticRegression()
prop_model.fit(X, T)
est_prop_score = prop_model.predict_proba(X)[:, 1]

treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

treatment_outcome_model.fit(X_treatment, Y_treatment)
control_outcome_model.fit(X_control, Y_control)

est_treatment_outcome = treatment_outcome_model.predict(X)
est_control_outcome = control_outcome_model.predict(X)

IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

# Evaluate performance
IPW_bias = IPW_est - true_ATE

print(f"Estimated ATE: {IPW_est}")
print(f"Bias: {IPW_bias}")

DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

# Evaluate performance
DR_bias = DR_est - true_ATE

print(f"Estimated ATE: {DR_est}")
print(f"Bias: {DR_bias}")

DM_est = np.mean(est_treatment_outcome - est_control_outcome)

# Evaluate performance
DM_bias = DM_est - true_ATE

print(f"Estimated ATE: {DM_est}")
print(f"Bias: {DM_bias}")

Estimated ATE: 2.887012893946858
Bias: -0.11298710605314222
Estimated ATE: 2.9749627896566637
Bias: -0.02503721034333628
Estimated ATE: 2.97911273489089
Bias: -0.020887265109109876


In [178]:
                                                              result_list = []

for tr in range(num_trial):
    result_list_temp = []
    
    
    X = np.random.normal(0, 1, (n, p))

    # Define a propensity score model
    # Assume treatment probability is a sigmoid function of a subset of covariates
    propensity_coef = np.random.normal(0, 0.5, p)
    propensity_scores = expit(X @ propensity_coef)  # Calculate propensity scores

    # Generate treatment assignment based on propensity scores
    T = np.random.binomial(1, propensity_scores)

    # Generate outcome with treatment effect
    # Assume a simple linear model for demonstration
    beta = np.random.normal(0, 1, p)
    Y = (X @ beta)**2 + 1.1 + treatment_effect * T + np.random.normal(0, 1, n)

    X_treatment = X[T == 1]
    X_control = X[T == 0]

    Y_treatment = Y[T == 1]
    Y_control = Y[T == 0]
    
        #### Direct bias correction
    prop_model = DirectBiasCorrection(lbd = 0.01)
    
    prop_model.fit(X, T, Y)
    est_prop_score = prop_model.predict_proba(X)[:, 1]
    est_prop_score_dbc = est_prop_score

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    
    ##### Linear models
    
    # Fit a linear model to estimate the treatment effect
    model = LinearRegression()
    model.fit(np.hstack([X, T.reshape(-1, 1)]), Y)
    estimated_treatment_effect = model.coef_[-1]

    # Evaluate performance
    true_ATE = treatment_effect
    bias = estimated_treatment_effect - true_ATE
    mse = mean_squared_error(Y, model.predict(np.hstack([X, T.reshape(-1, 1)])))

    print(f"Estimated ATE: {estimated_treatment_effect}")
    print(f"Bias: {bias}")
    print(f"Mean Squared Error: {mse}")

    result_list_temp.append(estimated_treatment_effect)
    
    
    
    #### Logistc regression 
    
    prop_model = LogisticRegression()
    prop_model.fit(X, T)
    est_prop_score = prop_model.predict_proba(X)[:, 1]

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    

    
    
    #### CBPS
    
    # Enable automatic conversion of Pandas DataFrame to R DataFrame
    pandas2ri.activate()

    # Simulate data in Python

    # Create a pandas DataFrame
    column_names = [f'X{i+1}' for i in range(p)]
    df = pd.DataFrame(X, columns=column_names)
    df['T'] = T
    df['Y'] = Y


    # Convert pandas DataFrame to R DataFrame
    r_df = pandas2ri.py2rpy(df)

    ro.r.assign("p", p)

    # Load the CBPS package in R and fit the model for ATE estimation
    ro.r('''
        library(CBPS)
        estimate_cbps_ate <- function(df) {
            formula_str <- paste("T ~", paste(names(df)[1:{p}], collapse=" + "))

            # CBPSの適用 (ATEの推定、ATT=0)
            model <- CBPS(as.formula(formula_str), data = df, ATT = 0, method = "exact")

            # 推定された傾向スコアの取得
            df$propensity_score <- fitted(model)

            # IPW (Inverse Probability Weighting) を適用
            df$weight <- ifelse(df$T == 1, 1 / df$propensity_score, 1 / (1 - df$propensity_score))

            # 重み付き回帰によるATEの推定
            result <- lm(Y ~ T, data = df, weights = df$weight)

            return(df$propensity_score)
        }
    ''')

    # R関数を呼び出してATEと傾向スコアを取得
    est_prop_score = ro.r['estimate_cbps_ate'](r_df)
    
    est_prop_score_cbps = est_prop_score
    
    #print(er)

    treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
    control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

    treatment_outcome_model.fit(X_treatment, Y_treatment)
    control_outcome_model.fit(X_control, Y_control)

    est_treatment_outcome = treatment_outcome_model.predict(X)
    est_control_outcome = control_outcome_model.predict(X)

    IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

    # Evaluate performance
    IPW_bias = IPW_est - true_ATE

    print(f"Estimated ATE: {IPW_est}")
    print(f"Bias: {IPW_bias}")
    
    result_list_temp.append(IPW_est)
    
    DM_est = np.mean(est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DM_bias = DM_est - true_ATE

    print(f"Estimated ATE: {DM_est}")
    print(f"Bias: {DM_bias}")
    
    result_list_temp.append(DM_est)

    DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

    # Evaluate performance
    DR_bias = DR_est - true_ATE

    print(f"Estimated ATE: {DR_est}")
    print(f"Bias: {DR_bias}")
    
    result_list_temp.append(DR_est)
    
    
    result_list.append(result_list_temp)

In [179]:
# Fit a linear model to estimate the treatment effect
prop_model = DirectBiasCorrection()
prop_model.fit(X, T)
est_prop_score = prop_model.predict_proba(X)[:, 1]

treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

treatment_outcome_model.fit(X_treatment, Y_treatment)
control_outcome_model.fit(X_control, Y_control)

est_treatment_outcome = treatment_outcome_model.predict(X)
est_control_outcome = control_outcome_model.predict(X)

IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

# Evaluate performance
IPW_bias = IPW_est - true_ATE

print(f"Estimated ATE: {IPW_est}")
print(f"Bias: {IPW_bias}")

DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

# Evaluate performance
DR_bias = DR_est - true_ATE

print(f"Estimated ATE: {DR_est}")
print(f"Bias: {DR_bias}")

DM_est = np.mean(est_treatment_outcome - est_control_outcome)

# Evaluate performance
DM_bias = DM_est - true_ATE

print(f"Estimated ATE: {DM_est}")
print(f"Bias: {DM_bias}")

Estimated ATE: 2.9685072990904686
Bias: -0.03149270090953138
Estimated ATE: 2.972925625090657
Bias: -0.027074374909342946
Estimated ATE: 2.97911273489089
Bias: -0.020887265109109876


In [14]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import pandas as pd
import numpy as np

# Enable automatic conversion of Pandas DataFrame to R DataFrame

# Simulate data in Python

# Create a pandas DataFrame
column_names = [f'X{i+1}' for i in range(p)]
df = pd.DataFrame(X, columns=column_names)
df['T'] = T
df['Y'] = Y


# Convert pandas DataFrame to R DataFrame
r_df = pandas2ri.py2rpy(df)

ro.r.assign("p", p)

# Load the CBPS package in R and fit the model for ATE estimation
ro.r('''
    library(CBPS)
    estimate_cbps_ate <- function(df) {
        formula_str <- paste("T ~", paste(names(df)[1:{p}], collapse=" + "))
        
        # CBPSの適用 (ATEの推定、ATT=0)
        model <- CBPS(as.formula(formula_str), data = df, ATT = 0, method = "exact")
        
        # 推定された傾向スコアの取得
        df$propensity_score <- fitted(model)
        
        # IPW (Inverse Probability Weighting) を適用
        df$weight <- ifelse(df$T == 1, 1 / df$propensity_score, 1 / (1 - df$propensity_score))
        
        # 重み付き回帰によるATEの推定
        result <- lm(Y ~ T, data = df, weights = df$weight)
        
        return(df$propensity_score)
    }
''')

# R関数を呼び出してATEと傾向スコアを取得
est_prop_score = ro.r['estimate_cbps_ate'](r_df)

treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

treatment_outcome_model.fit(X_treatment, Y_treatment)
control_outcome_model.fit(X_control, Y_control)

est_treatment_outcome = treatment_outcome_model.predict(X)
est_control_outcome = control_outcome_model.predict(X)

IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

# Evaluate performance
IPW_bias = IPW_est - true_ATE

print(f"Estimated ATE: {IPW_est}")
print(f"Bias: {IPW_bias}")

DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

# Evaluate performance
DR_bias = DR_est - true_ATE

print(f"Estimated ATE: {DR_est}")
print(f"Bias: {DR_bias}")

DM_est = np.mean(est_treatment_outcome - est_control_outcome)

# Evaluate performance
DM_bias = DM_est - true_ATE

print(f"Estimated ATE: {DM_est}")
print(f"Bias: {DM_bias}")

ModuleNotFoundError: No module named 'rpy2'

In [144]:
est_prop_score

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x326c98250> [3]
R classes: ('function',)

In [113]:
DM_est = np.mean(est_treatment_outcome - est_control_outcome)

# Evaluate performance
DM_bias = DM_est - true_ATE

print(f"Estimated ATE: {DM_est}")
print(f"Bias: {DM_bias}")

Estimated ATE: 2.9181776676318285
Bias: -0.08182233236817149


In [114]:


IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

# Evaluate performance
IPW_bias = IPW_est - true_ATE

print(f"Estimated ATE: {IPW_est}")
print(f"Bias: {IPW_bias}")

Estimated ATE: 3.067399653974786
Bias: 0.06739965397478587


In [115]:
T

array([1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,

In [116]:
import torch
import torch.nn as nn
import torch.optim as optim


KeyboardInterrupt



In [ ]:
# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
T_tensor = torch.tensor(T, dtype=torch.float32).view(-1, 1)

# Define a simple neural network model for propensity score estimation
class PropensityScoreNN(nn.Module):
    def __init__(self, input_dim):
        super(PropensityScoreNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Initialize model, loss function, and optimizer
model = PropensityScoreNN(p)
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the neural network
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_tensor)
    loss = criterion(outputs, T_tensor)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss for every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Estimated propensity scores for all data
with torch.no_grad():
    estimated_propensity_scores = model(X_tensor).numpy()

In [187]:
estimated_propensity_scores[estimated_propensity_scores < 0.01] = 0.01

In [188]:
estimated_propensity_scores[estimated_propensity_scores > 0.99] = 0.99

In [181]:
np.mean((T/estimated_propensity_scores.T[0] - (1-T)/(1 - estimated_propensity_scores.T[0]))*Y)

2.011218015922772

In [194]:
# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
T_tensor = torch.tensor(T, dtype=torch.float32).view(-1, 1)

# Define a simple neural network model for propensity score estimation
class PropensityScoreNN(nn.Module):
    def __init__(self, input_dim):
        super(PropensityScoreNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Initialize model, loss function, and optimizer
model = PropensityScoreNN(p)
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the neural network
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_tensor)
    outputs = torch.clamp(outputs, min=0.01, max=0.99)
    loss = -2*(1/outputs + 1/(1 - outputs)) + (T_tensor / outputs - (1-T_tensor) / (1-outputs))**2
    loss = loss.mean()
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss for every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Estimated propensity scores for all data
with torch.no_grad():
    estimated_propensity_scores = model(X_tensor).numpy()

Epoch [100/500], Loss: -89.3757
Epoch [200/500], Loss: -77.3542
Epoch [300/500], Loss: -126.3577
Epoch [400/500], Loss: -137.7103
Epoch [500/500], Loss: -146.9600


In [189]:
Z_tensor = torch.cat([T_tensor, X_tensor], axis=1)
Y_tensor = torch.tensor(Y, dtype=torch.float32).view(-1, 1)
dim = Z_tensor.shape[1]

class CodOutcomeNN(nn.Module):
    def __init__(self, input_dim):
        super(CodOutcomeNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    
# Initialize model, loss function, and optimizer
model = CodOutcomeNN(dim)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the neural network
num_epochs = 10000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(Z_tensor)
    loss = ((Y_tensor - outputs)**2).mean()
    loss = loss.mean()
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss for every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Estimated propensity scores for all data
with torch.no_grad():
    estimated_conditional_outcomes = model(Z_tensor).numpy()
    estimated_conditional_outcomes_1 = model(Z_tensor_1).numpy()
    estimated_conditional_outcomes_0 = model(Z_tensor_0).numpy()

Epoch [100/10000], Loss: 66.5176
Epoch [200/10000], Loss: 55.3388
Epoch [300/10000], Loss: 43.7221
Epoch [400/10000], Loss: 34.2139
Epoch [500/10000], Loss: 28.5578
Epoch [600/10000], Loss: 25.6987
Epoch [700/10000], Loss: 23.8395
Epoch [800/10000], Loss: 22.1396
Epoch [900/10000], Loss: 20.4078
Epoch [1000/10000], Loss: 18.6166
Epoch [1100/10000], Loss: 16.7452
Epoch [1200/10000], Loss: 14.7996
Epoch [1300/10000], Loss: 12.8643
Epoch [1400/10000], Loss: 10.9603
Epoch [1500/10000], Loss: 9.1128
Epoch [1600/10000], Loss: 7.3873
Epoch [1700/10000], Loss: 5.9171
Epoch [1800/10000], Loss: 4.7640
Epoch [1900/10000], Loss: 3.9100
Epoch [2000/10000], Loss: 3.3287
Epoch [2100/10000], Loss: 2.9629
Epoch [2200/10000], Loss: 2.7293
Epoch [2300/10000], Loss: 2.5687
Epoch [2400/10000], Loss: 2.4482
Epoch [2500/10000], Loss: 2.3496
Epoch [2600/10000], Loss: 2.2630
Epoch [2700/10000], Loss: 2.1825
Epoch [2800/10000], Loss: 2.1069
Epoch [2900/10000], Loss: 2.0346
Epoch [3000/10000], Loss: 1.9642
Epoch

In [190]:
with torch.no_grad():
    estimated_conditional_outcomes = model(Z_tensor).numpy()
    estimated_conditional_outcomes_1 = model(Z_tensor_1).numpy()
    estimated_conditional_outcomes_0 = model(Z_tensor_0).numpy()

In [191]:
aaa = np.mean(estimated_conditional_outcomes_1 - estimated_conditional_outcomes_0)

In [192]:
aaa

2.98163

In [193]:
aaa + np.mean((T/estimated_propensity_scores.T[0] - (1-T)/(1 - estimated_propensity_scores.T[0]))*(Y - estimated_conditional_outcomes.T[0]))

2.979416648378537

In [160]:
Y - estimated_conditional_outcomes.T[0]

array([ 5.63514307e-01,  4.77596318e-01,  6.29739734e-02, -1.27126940e-01,
       -1.77228238e-01,  4.64099619e-01, -3.51511405e-01, -2.99983419e-01,
       -2.69382789e-01,  1.26012672e-01, -3.40216497e-03,  2.07889849e-01,
       -6.60575762e-01, -4.97077542e-01, -8.41113774e-01, -1.26968687e-01,
       -4.25912768e-01, -9.33943451e-02, -4.43000561e-01,  2.44456645e-01,
        7.93688016e-02, -7.86699426e-01,  6.66016525e-01,  5.07777333e-01,
       -6.60204782e-01,  1.76161963e-01,  3.64499704e-01, -2.11489539e-01,
       -3.55552370e-01, -6.61527685e-01, -7.72481761e-02, -2.88697698e-01,
       -6.51645266e-01, -1.14941385e+00, -5.81560010e-01,  9.66436085e-01,
       -7.85939945e-01, -4.66742301e-01,  7.49621729e-01,  2.88066177e-01,
       -5.62568961e-01, -1.75046911e-01,  2.87174176e-01,  5.13794861e-01,
       -3.57863637e-01, -6.20851911e-01, -4.15798049e-01, -4.12468909e-01,
        2.46813750e-01, -5.61748112e-01,  7.15916546e-01, -6.73407462e-01,
       -1.76732254e-01, -